In [ ]:
import guidance
from guidance import models, gen, one_or_more, select, zero_or_more, regex, optional, capture

model_name = "your_model_name_here"

model = models.LlamaCpp(f"{model_name}.gguf", n_gpu_layers=-1, n_ctx=2048)

llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [ ]:
@guidance(stateless=False)
def drinkOrderCoffee(lm):
    lm += "DrinkOrder("
    lm += select(["number="+regex("\d+"), ""], name='numberFlag')
    if drink_type_values:
      lm += select([", drink_type='" + select(drink_type_values, name='drinkTypeName')+"'", ""], name='drinkTypeFlag')
      if lm['drinkTypeFlag'] != "":
        drink_type_values.remove(lm['drinkTypeName'])

    if roast_type_values:
      lm += select([", roast_type='"+select(roast_type_values, name='roastTypeName')+"'", ""], name='drinkTypeFlag')
      if lm['drinkTypeFlag'] != "":
        roast_type_values.remove(lm['roastTypeName'])

    if size_values:
      lm += select([", size='"+select(size_values, name='sizeName')+"'", ""], name='sizeFlag')
      if lm['sizeFlag'] != "":
        size_values.remove(lm['sizeName'])

    if style_values:
      lm += select([", style='"+select(style_values, name='styleName')+"'", ""], name='styleFlag')
      if lm['styleFlag'] != "":
        style_values.remove(lm['styleName'])

    if topping_values:
      lm += select([", toppings=[", ""], name='toppingsFlag')
      if lm['toppingsFlag']:
        for i in topping_values[:]:
          lm += toppingCoffee()
          if not topping_values:
            lm += ']'
            break
          lm += select([", ", "]"], name="finishedListToppings")
          if lm['finishedListToppings'] == "]":
            break



    return lm + ")"

@guidance(stateless=False)
def toppingCoffee(lm):
  lm += "Topping(name="
  if topping_values:
    lm += "'" + select(topping_values, name='toppingName') + "'"
    topping_values.remove(lm['toppingName'])

  if quantity_values:
    lm += select([", qualifier='" + select(quantity_values, name='qualifierName') + "'", ""], name='qualifierFlag')
    if lm["qualifierFlag"] != "":
      quantity_values.remove(lm['qualifierName'])

  if not_values:
    lm += select([", negation=True", ""], name='negationFlag')
    if lm['negationFlag'] != "":
      not_values.remove('not')

  lm += ")"
  return lm

@guidance(stateless=False)
def validOrderCoffee(lm):
  lm += "["
  first = True
  for i in range(7):
    if drink_type_values:
      if not first:
        lm += select(", ", "")
      else:
        first = False

      lm += drinkOrderCoffee()
    else:
      break
  return lm +']'

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_134591/2586245235.py:4: SyntaxWarning: invalid escape sequence '\d'
  lm += select(["number="+regex("\d+"), ""], name='numberFlag')
<unknown>:4: SyntaxWarning: invalid escape sequence '\d'


In [3]:
instruction_generate_coffee = """You are a helpful assistant. You have to take as input a customer order and output a list of the corresponding objects. You should use only the following classes in Python:
class Topping:
      def __init__(self, name: str, qualifier: Optional[str] = None, negation: Optional[bool] = False) -> None:

class DrinkOrder:
      def __init__(self, number: int = 1, drink_type: Optional[str] = None, size: Optional[str] = None, style: Optional[str] = None, roast_type: Optional[str] = None, toppings: Optional[List[Topping]] = None) -> None:

The output should be a list of those objects."""


In [ ]:
import json
import spacy
from spacy.pipeline import EntityRuler
from spacy import displacy
import os
import re

def parse_line(line):
    parts = line.strip().split('\t')
    if len(parts) != 2:
        return None, None

    phrase, category = parts

    return phrase, category.strip()

def init_pipeline(dataset = "coffee"):
  nlp = spacy.load("en_core_web_sm")
  ner_ruler = nlp.add_pipe("entity_ruler",
                      before="ner",
                      config={"phrase_matcher_attr": "LOWER"})

  def read_file_categories(food_type):
      file_path = f"FoodOrderingDataset/data/{food_type}/alias"

      text_files = [f for f in os.listdir(file_path) if f.endswith('.txt')]

      patterns = []

      for file in text_files:
          path_to_file = f"{file_path}/{file}"
          with open(path_to_file, 'r') as file:
              for line in file:
                  if line.strip():
                      phrase, category_info = parse_line(line)
                      if phrase and category_info:
                          schema = {}
                          schema["pattern"] = phrase
                          schema["label"] = category_info
                          patterns.append(schema)

      return patterns

  category_patterns = read_file_categories(dataset)

  ner_ruler.add_patterns(category_patterns)
  return nlp


nlp = init_pipeline()

def process_NER(input_order):
    found_categories = []

    doc = nlp(input_order)

    for ent in doc.ents:
        found_categories.append((ent.text, ent.label_))

    return found_categories

In [ ]:
import json
import re
from collections import defaultdict

file_path = f'FoodOrderingDataset/output/{model_name}-coffee-NER.json'

existing_data=[]

with open('FoodOrderingDataset/processed_data/coffee_dataset.json', 'r') as file:
    data = json.load(file)

input_list = []
for obj in data:
    input_value = obj.get("input", "No input key found")
    output_value = obj.get("output_extract", "No output key found")
    output_generate = obj.get("output_generate", "No output key found")
    used_items_value = process_NER(input_value)
    used_items_value_decoupled = [x[0] + ' - ' + x[1] for x in used_items_value]
    used_items_str = ', '.join(used_items_value_decoupled).lower()

    input_augmented_file = input_value + "\nItems Found: " + used_items_str
    input_list.append((input_value, input_augmented_file, output_generate, used_items_value, used_items_str))

for i in range(len(input_list)):
    if i > 130:
        break
    (initial_input, input, expected, used_items_value, used_items_str) = input_list[i]
    lm = model + f'''\
    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    ### Instruction:
    {instruction_generate_coffee}
    ### Input:
    {input}

    ### Response:
    '''

    items = []
    print(used_items_value)
    for item in used_items_value:
      match = re.search(r'(\w+)\(([^)]+)\)', item[1])
      if match:
        items.append((match.group(1), match.group(2)))
    items_dict = defaultdict(list)
    for item_type, item_value in items:
        items_dict[item_type.lower()].append(item_value.lower())

    print(items_dict)
    items_dict = dict(items_dict)
    keys_to_extract = ['topping', 'size', 'number', 'drink_type', 'roast_type', 'not', 'style', 'quantity']

    for key in keys_to_extract:
        globals()[f"{key}_values"] = items_dict.get(key, [])
        print(key, globals()[f"{key}_values"])

    ans = lm + capture(validOrderCoffee(), "answer")

    existing_data.append({"input":initial_input, "input_augmented": input, "output": ans["answer"], "expected":expected, "output_NER": used_items_str})

with open(file_path, 'w') as file:
    json.dump(existing_data, file, indent=4)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

[('a', 'number(1)'), ('regular', 'SIZE(regular)'), ('latte', 'DRINK_TYPE(latte)'), ('cinnamon', 'TOPPING(cinnamon)'), ('iced', 'STYLE(iced)'), ('one extra espresso shot', 'TOPPING(ESPRESSO_SHOT_1)')]
defaultdict(<class 'list'>, {'number': ['1'], 'size': ['regular'], 'drink_type': ['latte'], 'topping': ['cinnamon', 'espresso_shot_1'], 'style': ['iced']})
topping ['cinnamon', 'espresso_shot_1']
size ['regular']
number ['1']
drink_type ['latte']
roast_type []
not []
style ['iced']
quantity []


/tmp/ipykernel_134591/2586245235.py:4: SyntaxWarning: invalid escape sequence '\d'
  lm += select(["number="+regex("\d+"), ""], name='numberFlag')


KeyboardInterrupt: 

In [ ]:
import json
import re

def remove_duplicate_toppings(order_str: str) -> str:
    pattern = r"(toppings=\[)(.*?)(\])"
    
    def dedupe(match: re.Match) -> str:
        prefix = match.group(1)
        content = match.group(2)
        suffix = match.group(3)
        
        topping_pattern = r"Topping\([^)]*\)"
        topping_items = re.findall(topping_pattern, content)
        
        seen = set()
        unique_toppings = []
        for item in topping_items:
            if item not in seen:
                seen.add(item)
                unique_toppings.append(item)
        
        new_content = ", ".join(unique_toppings)
        return f"{prefix}{new_content}{suffix}"
    
    cleaned_str = re.sub(pattern, dedupe, order_str, flags=re.DOTALL)
    if cleaned_str[-4:] != ")])]" and "Topping" in cleaned_str: cleaned_str += ")]"
    return cleaned_str

In [ ]:
import json

def calculate_accuracy_and_save_mismatches(json_file, output_file):
    with open(json_file, 'r') as file:
        data = json.load(file)

    total = len(data)
    correct = 0
    mismatches = []

    for item in data:
        output = item['output']
        expected = item['expected'].lower()

        output = remove_duplicate_toppings(output).lower()

        if output == expected:
            correct += 1
        else:
            print('\n\n')
            print(output)
            print(expected)
            mismatches.append(item)

    accuracy = (correct / total) * 100

    with open(output_file, 'w') as outfile:
        json.dump(mismatches, outfile, indent=4)

    return accuracy

json_file = f'FoodOrderingDataset/output/{model_name}-coffee-NER.json'
mismatch_file = f'FoodOrderingDataset/output/{model_name}-coffee-NER_processed_mismatches.json'

accuracy = calculate_accuracy_and_save_mismatches(json_file, mismatch_file)
print(f"Accuracy: {accuracy:.2f}%")
print(f"Mismatches have been saved to: {mismatch_file}")




[drinkorder(number=1, drink_type='latte', roast_type='light_roast', size='large', toppings=[topping(name='honey'), topping(name='espresso_shot_1'), topping(name='caramel_syrup')]), drinkorder(number=1, drink_type='cappuccino', size='regular')])]
[drinkorder(number=1, drink_type='latte', roast_type='light_roast', size='regular', toppings=[topping(name='espresso_shot_1'), topping(name='honey')]), drinkorder(number=1, drink_type='cappuccino', size='large', toppings=[topping(name='caramel_syrup')])]



[drinkorder(number=1, drink_type='latte', roast_type='french', size='large', toppings=[topping(name='caramel_syrup')]), drinkorder(number=1, drink_type='americano', roast_type='dark_roast', size='regular')])]
[drinkorder(number=1, drink_type='latte', roast_type='french', size='large'), drinkorder(number=1, drink_type='americano', roast_type='dark_roast', size='regular', toppings=[topping(name='caramel_syrup')])]



[drinkorder(number=1, drink_type='latte', roast_type='dark_roast', size='l